In [14]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import optimizers
import matplotlib.pyplot as plt

In [17]:
# Define the folder where the RBC database is going to be saved
# -------------------------------------------------------------

base_dir = './RBC_database'

# Create a folder for the training, validation and testing data. 
# For each of them, two folders are created, one for the good RBC
# images, and the other for the bad ones.
# --------------------------------------

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_goodRBC_dir = os.path.join(train_dir,'goodRBC')
train_badRBC_dir = os.path.join(train_dir,'badRBC')

validation_goodRBC_dir = os.path.join(validation_dir,'goodRBC')
validation_badRBC_dir = os.path.join(validation_dir,'badRBC')

test_goodRBC_dir = os.path.join(test_dir,'goodRBC')
test_badRBC_dir = os.path.join(test_dir,'badRBC')

In [18]:
# Preprocess the images. The idea here is to create the mini-bacth of images using an
# image generator. Also the images are normalized in such way the maximum intensity is
# equal to 1. 
# ----------

test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        horizontal_flip = True,
        vertical_flip = True)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(85,85),
        batch_size = 50,
        class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(85,85),
        batch_size = 50,
        class_mode = 'binary')

Found 2956 images belonging to 2 classes.
Found 993 images belonging to 2 classes.


In [28]:
# Define the model and the compilation options. In that case, the VGG model is loaded, keeping only the 
# convolution part of the model. Finally, the weights of the VGG model are kept and set to "not trainable"
# --------------------------------------------------------------------------------------------------------

from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',include_top = False,input_shape=(85,85,3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

conv_base.trainable = False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_6 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_17 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
__________

In [26]:
# Define as trainable the last convolution block as well as the densely connected layers. The routine below
# is changing the trainable properties of the network from True to False for all the layers except the last 
# ones composing the block5.
# --------------------------

conv_base.trainable = False

for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if layer.name == 'block5_conv2':
        set_trainable = True
    if layer.name == 'block5_conv3':
        set_trainable = True 
            
model.compile(optimizer = optimizers.RMSprop(lr=1e-5), 
            loss='binary_crossentropy',
            metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
__________

In [27]:
# Train the model
# ---------------

history = model.fit_generator(train_generator,
                    steps_per_epoch = 100,
                    epochs = 30,
                    validation_data = validation_generator,
                    validation_steps = 50)

# Save the model
# --------------

model.save('RBC_classification_VGG16_1.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
  3/100 [..............................] - ETA: 9:30 - loss: 0.7982 - acc: 0.6267 

KeyboardInterrupt: 

In [ ]:
# Display the loss function during the training
# ---------------------------------------------

history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

n = len(loss_values)
epochs = range(1, n+1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# The accuracy of the model is tested using the testing set of data.
# ------------------------------------------------------------------

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

n = len(acc_values)
epochs = range(1, n+1)

plt.plot(epochs, acc_values, 'bo', label='Training acccuracy')
plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()

plt.show()


In [ ]:
# Finally test the accuracy of the model directly on the testing set of images
# ----------------------------------------------------------------------------

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(85,85),
        batch_size = 20,
        class_mode = 'binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps = 50)
print('test_acc : ', test_acc)
print('test_loss : ', test_loss)